## Set Environment

In [1]:
# !pip install opencv-python
import sys
import numpy as np
import math
import os
import cv2

## Global Variable

In [2]:
Q_table = np.array(
[[16,  11,  10,  16,  24,  40,  51,  61],
[12,  12,  14,  19,  26,  58,  60,  55],
[14,  13,  16,  24,  40,  57,  69,  56],
[14,  17,  22,  29,  51,  87,  80,  62],
[18,  22,  37,  56,  68, 109, 103,  77],
[24,  35,  55,  64,  81, 104, 113,  92],
[49,  64,  78,  87, 103, 121, 120, 101],
[72,  92,  95,  98, 112, 100, 103,  99]]).astype(np.float)

# control quality factor
QF = 70.0
scale = 200.0 - 2 * QF
scale /= 100.0
Q_table *= scale

grid_sz = 8

## Functions

In [3]:
# input: Basis (B), 2D image array (X)
# output: coefficient in 2D (a)
# X = B @ a @ B_transpose
# => a = B_inverse @ X @ B_transepose_inverse
# = B_inverse @ X @ B_inverse_transpose
def CosineTrans2d(B, X):
    # implement 2D DCT
    inv_B = np.linalg.inv(B)
    tmp = inv_B @ X
    a = tmp @ (inv_B.T)
    return a

# input: Basis (B), coefficient in 2D (a)
# output: reconstructed image (X')
def InvCosineTrans2d(B, a):
    # implement 2D iDCT
    tmp = B @ a
    X = tmp @ (B.T)
    return X

def gen_basis(N):
    # Cosine Transformation
    C = math.sqrt(2)/math.sqrt(N)
    PI = math.pi
    B = np.array([[1/math.sqrt(N) for i in range(N)]])
    
    for k in range(1, N):
        b = []
        for n in range(N):
            elem = C * math.cos((n + 0.5) * k * PI / N)
            b.append(elem)
        B = np.vstack([B, b])
    return B.T

'''
function : compress_grid
---
2D DCT -> coefficient -> inverse 2D DCT -> reconstructed and compressed image 
'''
def compress_grid(X):
    count = 0
    N = len(X)
    
    # generate basis "B" and compute variable "DCT" 
    #coefficient of grid I  
    B = gen_basis(N)
    DCT = CosineTrans2d(B, X)
  
    # make a copy for DCT
    DCT_chopped = DCT.copy()
    # quantize DCT given pre-defined Quantization matrix 
    # and do rounding 
    DCT_quantized = np.around(DCT / Q_table)
  
    for x in range(N):
        for u in range(N):
            # convert the remaining DCT value
            DCT_chopped[x,u] = DCT_quantized[x, u] * Q_table[x, u]
            # count the number of values in matrix that has been set to 0
            if DCT_chopped[x,u] == 0.0:
                count += 1
            
    # do inverse 2D DCT on "DCT_chopped" to reconstruct the grid, save as "reconstruct_I"
    reconstruct_X = InvCosineTrans2d(B, DCT_chopped)

    return reconstruct_X + 128, count

## Load Data

In [4]:
I = cv2.imread('test.png', cv2.IMREAD_GRAYSCALE).astype('float')
# centralize
I = I - 128
N = len(I)
reconstruct_I = np.zeros((N, N))
tot_count = 0 

In [5]:
I.shape

(512, 512)

In [6]:
# crop original image to 8*8 grid and do JPEG compression for each grid
for r in range(N // grid_sz):
    for c in range(N // grid_sz):
        grid = I[r*grid_sz:r*grid_sz+grid_sz, c*grid_sz:c*grid_sz+grid_sz]
        print(grid)
        reconstruct_chopped, count = compress_grid(grid)
        tot_count += count
        reconstruct_I[r*grid_sz:r*grid_sz+grid_sz, c*grid_sz:c*grid_sz+grid_sz] = reconstruct_chopped
print(f'JPEG compression: only use {1 - tot_count / (N * N)} size of original image')

[[34. 34. 34. 33. 34. 29. 35. 33.]
 [34. 34. 34. 33. 34. 29. 35. 33.]
 [34. 34. 34. 33. 34. 29. 35. 33.]
 [34. 34. 34. 33. 34. 29. 35. 33.]
 [34. 34. 34. 33. 34. 29. 35. 33.]
 [36. 36. 30. 27. 33. 31. 31. 32.]
 [32. 32. 35. 30. 32. 34. 31. 28.]
 [31. 31. 27. 29. 30. 31. 28. 29.]]
[[37. 33. 34. 32. 27. 35. 32. 27.]
 [37. 33. 34. 32. 27. 35. 32. 27.]
 [37. 33. 34. 32. 27. 35. 32. 27.]
 [37. 33. 34. 32. 27. 35. 32. 27.]
 [37. 33. 34. 32. 27. 35. 32. 27.]
 [32. 32. 27. 31. 26. 26. 28. 26.]
 [31. 34. 28. 34. 27. 27. 28. 24.]
 [31. 33. 33. 30. 25. 25. 25. 23.]]
[[29. 28. 33. 33. 26. 28. 26. 29.]
 [29. 28. 33. 33. 26. 28. 26. 29.]
 [29. 28. 33. 33. 26. 28. 26. 29.]
 [29. 28. 33. 33. 26. 28. 26. 29.]
 [29. 28. 33. 33. 26. 28. 26. 29.]
 [28. 28. 25. 29. 24. 25. 25. 22.]
 [25. 32. 26. 26. 30. 22. 27. 24.]
 [27. 26. 28. 25. 27. 25. 24. 24.]]
[[27. 29. 27. 24. 28. 26. 26. 28.]
 [27. 29. 27. 24. 28. 26. 26. 28.]
 [27. 29. 27. 24. 28. 26. 26. 28.]
 [27. 29. 27. 24. 28. 26. 26. 28.]
 [27. 29. 27. 24.

 [19. 22. 24. 24. 27. 26. 26. 25.]]
[[34. 31. 35. 33. 33. 28. 30. 31.]
 [33. 30. 33. 32. 28. 31. 31. 30.]
 [28. 34. 36. 33. 31. 29. 31. 30.]
 [30. 30. 30. 32. 30. 28. 30. 28.]
 [29. 26. 33. 31. 32. 29. 29. 30.]
 [30. 29. 31. 26. 28. 28. 28. 32.]
 [28. 31. 31. 32. 28. 33. 29. 30.]
 [27. 28. 28. 30. 28. 34. 32. 32.]]
[[32. 30. 30. 31. 32. 33. 30. 33.]
 [29. 30. 28. 33. 34. 31. 34. 31.]
 [32. 29. 28. 30. 31. 31. 28. 31.]
 [30. 30. 27. 30. 30. 29. 29. 32.]
 [29. 25. 30. 29. 31. 28. 32. 31.]
 [26. 26. 30. 28. 29. 29. 32. 29.]
 [29. 28. 31. 30. 27. 28. 29. 28.]
 [28. 27. 30. 28. 29. 28. 28. 27.]]
[[30. 28. 33. 30. 30. 27. 19. 31.]
 [32. 31. 29. 34. 31. 31. 30. 27.]
 [32. 27. 30. 32. 30. 29. 28. 27.]
 [29. 29. 25. 28. 27. 29. 24. 32.]
 [31. 29. 28. 28. 28. 29. 24. 31.]
 [32. 29. 29. 27. 29. 27. 26. 28.]
 [26. 28. 32. 26. 26. 30. 27. 28.]
 [28. 31. 33. 25. 29. 27. 30. 25.]]
[[30. 31. 30. 29. 29. 28. 29. 30.]
 [27. 31. 26. 28. 30. 29. 26. 31.]
 [30. 29. 31. 28. 27. 27. 28. 28.]
 [32. 32. 28. 29

 [-25. -24. -27. -24. -28. -31. -34. -29.]]
[[-28. -21. -22. -18. -16. -10.   0.   3.]
 [-27. -24. -23. -18. -10.  -8.  -4.   5.]
 [-25. -28. -22. -17. -19. -14.  -1.   2.]
 [-32. -24. -20. -16. -12. -10.  -4.   7.]
 [-27. -15. -19. -16. -12. -12.  -2.   5.]
 [-30. -24. -21. -19. -12.  -9.   6.   3.]
 [-21. -22. -24. -17. -19.  -9.   2.   1.]
 [-26. -24. -20. -19. -20.  -9.  -3.   5.]]
[[ 7. 12. 17. 17. 18. 21. 24. 28.]
 [ 7. 15. 16. 19. 19. 21. 25. 28.]
 [ 7. 19. 20. 20. 23. 26. 24. 26.]
 [11. 19. 24. 22. 20. 26. 25. 26.]
 [11. 18. 18. 22. 23. 26. 25. 29.]
 [ 8. 14. 16. 18. 23. 25. 23. 29.]
 [11. 16. 16. 21. 21. 21. 20. 30.]
 [ 9. 10. 18. 20. 23. 21. 22. 30.]]
[[25. 25. 26. 25. 24. 25. 21. 20.]
 [24. 25. 24. 26. 29. 30. 24. 24.]
 [23. 26. 32. 30. 25. 29. 24. 26.]
 [25. 28. 27. 29. 30. 33. 28. 27.]
 [27. 29. 26. 27. 32. 27. 29. 25.]
 [28. 27. 29. 29. 30. 27. 26. 24.]
 [32. 28. 28. 26. 29. 28. 27. 27.]
 [27. 28. 27. 24. 29. 29. 24. 25.]]
[[24. 20. 20. 15. 18. 15. 15. 11.]
 [26. 21. 20. 

 [71. 62. 64. 68. 65. 69. 72. 69.]]
[[77. 79. 79. 75. 69. 70. 75. 83.]
 [80. 77. 72. 73. 72. 77. 75. 76.]
 [76. 72. 75. 70. 73. 74. 77. 72.]
 [75. 75. 76. 75. 71. 69. 78. 76.]
 [76. 76. 77. 75. 73. 67. 71. 76.]
 [70. 78. 76. 73. 70. 67. 64. 63.]
 [70. 71. 75. 72. 62. 59. 60. 65.]
 [70. 66. 60. 62. 59. 66. 68. 73.]]
[[77. 80. 76. 73. 80. 82. 79. 75.]
 [76. 76. 77. 75. 73. 77. 78. 72.]
 [71. 77. 79. 75. 72. 68. 69. 68.]
 [71. 70. 71. 70. 66. 69. 70. 71.]
 [70. 63. 60. 69. 75. 77. 80. 74.]
 [64. 66. 68. 77. 80. 78. 81. 77.]
 [68. 71. 73. 75. 80. 79. 77. 78.]
 [72. 72. 73. 72. 76. 76. 79. 78.]]
[[76. 75. 63. 61. 66. 71. 73. 74.]
 [69. 71. 64. 64. 70. 77. 77. 76.]
 [71. 75. 75. 73. 72. 76. 75. 75.]
 [75. 83. 82. 79. 76. 70. 70. 77.]
 [72. 80. 85. 82. 79. 72. 69. 72.]
 [73. 74. 79. 80. 80. 78. 72. 67.]
 [79. 71. 73. 79. 81. 79. 80. 75.]
 [80. 73. 73. 75. 79. 79. 77. 78.]]
[[74. 77. 79. 86. 84. 83. 81. 78.]
 [77. 79. 79. 81. 82. 85. 83. 79.]
 [76. 79. 79. 76. 76. 82. 85. 83.]
 [77. 79. 79. 80

 [27. 30. 32. 34. 37. 29. 28. 28.]]
[[30. 29. 27. 29. 28. 30. 30. 28.]
 [29. 29. 27. 26. 32. 30. 29. 26.]
 [27. 29. 30. 30. 27. 30. 30. 30.]
 [29. 28. 29. 29. 28. 28. 30. 32.]
 [28. 28. 29. 27. 28. 31. 30. 27.]
 [27. 30. 29. 29. 28. 33. 28. 26.]
 [28. 33. 31. 33. 29. 30. 30. 29.]
 [30. 28. 29. 33. 31. 31. 33. 31.]]
[[29. 24. 25. 24. 27. 25. 28. 27.]
 [29. 23. 28. 29. 26. 27. 27. 25.]
 [29. 27. 25. 29. 29. 28. 29. 27.]
 [27. 27. 28. 25. 30. 25. 26. 29.]
 [28. 30. 28. 27. 29. 27. 27. 28.]
 [30. 28. 30. 27. 31. 26. 25. 28.]
 [31. 30. 27. 30. 32. 31. 29. 25.]
 [26. 29. 30. 29. 29. 30. 29. 29.]]
[[27. 32. 26. 26. 27. 25. 26. 28.]
 [28. 23. 23. 23. 26. 30. 25. 24.]
 [23. 30. 24. 26. 26. 29. 29. 25.]
 [26. 28. 24. 29. 28. 32. 30. 27.]
 [26. 28. 24. 30. 28. 29. 28. 29.]
 [28. 28. 28. 25. 27. 29. 25. 26.]
 [27. 28. 28. 31. 29. 31. 27. 29.]
 [27. 29. 29. 29. 30. 32. 27. 28.]]
[[-33. -33. -30. -35. -34. -30. -30. -26.]
 [-29. -29. -33. -32. -29. -29. -33. -25.]
 [-33. -33. -29. -29. -25. -27. -31

...

 [42. 36. 31. 29. 21. 16.  3. -3.]]
[[-17. -28. -41. -49. -55. -53. -48. -49.]
 [-19. -34. -44. -52. -57. -57. -46. -45.]
 [-16. -32. -46. -49. -52. -55. -51. -44.]
 [-20. -34. -42. -45. -53. -56. -46. -46.]
 [-16. -34. -38. -54. -53. -48. -52. -40.]
 [-16. -30. -42. -50. -54. -49. -48. -46.]
 [-20. -29. -43. -48. -54. -50. -52. -43.]
 [-21. -25. -46. -50. -52. -56. -56. -46.]]
[[-44. -35. -42. -35. -30. -28. -28. -24.]
 [-42. -37. -36. -34. -29. -35. -29. -27.]
 [-43. -40. -35. -38. -30. -32. -32. -27.]
 [-41. -42. -38. -37. -28. -34. -33. -27.]
 [-42. -38. -30. -42. -33. -35. -28. -28.]
 [-44. -41. -38. -38. -36. -33. -30. -26.]
 [-43. -35. -31. -32. -30. -31. -31. -21.]
 [-39. -45. -39. -33. -33. -31. -30. -25.]]
[[-27. -18.  -6.  -2. -14. -15. -27. -40.]
 [-21. -10.  -6.  -7. -11.  -9. -25. -38.]
 [-23. -11.  -9. -16. -13. -10. -20. -31.]
 [-21. -17. -12. -14. -14.  -5.   1. -12.]
 [-27. -25. -22. -15.  -7.  16.  16.   2.]
 [-23. -25. -16.  -8.   5.  21.  26.   7.]
 [-24. -18. -10.

 [ 9. 10. 13. 13.  6. 12. 15.  9.]]
[[ 5.  5.  6. -2. -3. -6. -5. -6.]
 [ 7.  5.  6.  0.  0. -3. -1. -6.]
 [11.  4.  5.  5.  3.  3.  0. -5.]
 [ 5.  6.  3.  4.  6.  2.  1.  2.]
 [ 5.  7.  5.  5.  1.  7.  3.  1.]
 [11.  8.  6.  7.  4.  5.  2.  2.]
 [ 4.  7.  7.  8.  5.  1.  1.  4.]
 [ 7.  7. 12.  4.  8.  6.  5.  5.]]
[[ -6.  -1.  -1.   4.   2.   4.   1.  -3.]
 [ -2.  -6.  -4.  -1.   2.  -2.   3.  -2.]
 [  1.  -3.  -2.  -3.  -6.  -7.  -2.  -2.]
 [  3.   0.  -4.  -2.  -5.  -6.  -9. -10.]
 [  7.   5.   0.  -1.   3.  -6. -12. -11.]
 [  2.   3.  -1.  -1.  -2.  -3.  -1.  -3.]
 [  5.   1.   1.   2.   6.   5.   1.   5.]
 [ 10.   3.   1.   5.   6.   5.   8.  13.]]
[[ -2.   1.  12.  11.  13.  17.  15.  14.]
 [  6.  -3.   4.   9.  10.  11.  14.   9.]
 [ -2.  -3.   1.   8.   7.   4.   7.   9.]
 [ -8.  -5.  -4.   0.   4.  -1.   1.   7.]
 [ -9. -13. -11.  -9.  -7. -10.  -9.  -3.]
 [ -1. -11. -11. -12. -11. -13.  -9.  -9.]
 [  0.  -2.  -7.  -8. -11. -10.  -6. -11.]
 [  4.   9.   7.   1.   1.   0.   6. 

 [-63. -49. -61. -71. -48. -79. -37.  11.]]
[[ 35.  50.  24.  26.  32.  18.  -2.  -6.]
 [ 42.  45.  18.  26.  25.  16.  -2.  -6.]
 [ 50.  42.  20.  27.  22.  10.   1.  -8.]
 [ 54.  39.  19.  25.  20.   7.   0. -13.]
 [ 54.  35.  22.  23.  19.   5.  -1.  -6.]
 [ 53.  33.  21.  21.  22.   3.  -2.   1.]
 [ 51.  21.  24.  22.  19.   4.  -3.   5.]
 [ 51.  26.  26.  12.  19.   9.  -2.  13.]]
[[-13. -23.  -6. -18.  -5.  -7. -17. -26.]
 [ -6. -11.  -1. -12.  -6.   2.  -1.  -7.]
 [ -2. -11.   6.  -2.  -3.   0.   1.  -5.]
 [ -4.  -6.  11.   2.   1.   3.   6.   1.]
 [  2.   0.  11.   5.   4.   3.   6.  11.]
 [ -2.   6.  15.   3.   1.   4.   5.  12.]
 [  3.   9.  11.   7.   4.   4.   5.  12.]
 [  9.  11.  12.   8.   6.  11.   9.  10.]]
[[-16. -19. -15. -12. -17. -19. -10. -14.]
 [-14. -15. -13. -15. -17. -17.  -9. -11.]
 [-11. -12.  -7.  -9. -15. -13. -13. -13.]
 [ -9.  -9.  -3.  -9.  -6. -12. -12. -12.]
 [  4.   0.  -1.  -6.  -1.  -2.  -5.  -8.]
 [ 10.   6.   1.  -4.  -2.  -3.   1.  -9.]
 [ 17.  

[[-49. -56. -76. -44. -24. -57. -32. -18.]
 [-49. -53. -67. -42. -30. -61. -39. -14.]
 [-57. -58. -73. -56. -32. -62. -52. -18.]
 [-63. -61. -63. -67. -58. -63. -72. -28.]
 [-66. -54. -66. -62. -75. -78. -76. -58.]
 [-67. -44. -64. -72. -57. -59. -76. -62.]
 [-82. -45. -55. -75. -76. -68. -68. -28.]
 [-87. -49. -34. -69. -64. -72. -66. -35.]]
[[-12. -14. -27. -37. -50. -46. -41. -53.]
 [ -5. -10. -20. -50. -60. -53. -59. -58.]
 [  8.  -7. -12. -36. -70. -74. -71. -68.]
 [ -3.   6.  -1. -28. -68. -76. -71. -71.]
 [-19.   7.   6. -18. -59. -82. -62. -58.]
 [-25.  -7.  -1. -15. -41. -67. -64. -50.]
 [ -4. -16. -12. -17. -20. -31. -45. -43.]
 [ -8. -18. -23. -17.  -9.  -6. -10. -23.]]
[[-77. -52. -30.  -8.  25.  26.   2. -13.]
 [-70. -65. -47. -11.  36.  24.   1.   6.]
 [-73. -67. -52.   1.  33.  13.  11.   9.]
 [-64. -57. -42.   9.  27.  11.  11.  10.]
 [-43. -49. -34.   8.  12.  13.  13.  11.]
 [-25. -31. -30.  11.  10.   8.   7.  11.]
 [-21. -38. -32.  10.   9.  12.   8.  12.]
 [-15. -2

 [-49. -59. -75. -57. -67. -73. -68. -76.]]
[[-29. -19. -19. -15.   7.  25.  19.  -8.]
 [-61. -36. -22. -24. -17.  24.  38.  19.]
 [-91. -68. -36. -25. -24.   3.  29.  37.]
 [-79. -90. -79. -46. -27.   0.  11.  19.]
 [-79. -92. -92. -77. -51. -24.   3.   1.]
 [-87. -84. -87. -86. -67. -56. -20.  10.]
 [-85. -79. -79. -85. -60. -60. -61. -15.]
 [-81. -84. -75. -75. -60. -64. -61. -58.]]
[[-28. -10.  18.   8. -21. -18. -14. -48.]
 [ -7. -26. -14.  10.   5. -27. -39. -31.]
 [ 16.   0. -15.   5.  13. -11. -55. -39.]
 [ 18.  15.   4.   4.   2. -14. -48. -47.]
 [ -9.  18.  14.   2.   6.  -8. -24. -36.]
 [ -9.   3.  20.  22.  28.  15. -13. -23.]
 [  1.  -3.   3.  34.  43.  33.   5. -13.]
 [-35. -12.   0.  13.  24.  29.  19.   6.]]
[[-40. -33. -71. -61. -18. -16. -42. -60.]
 [-51. -30. -38. -53. -29. -34. -50. -53.]
 [-30. -34.  -7. -12.  -4. -16. -43. -64.]
 [-27. -14.  -4. -19.  10.  12. -23. -47.]
 [-33.  -6. -27. -65. -35.   2.  -8. -20.]
 [-15. -23. -29. -35. -41. -17.  -3. -10.]
 [-37. -

In [7]:
outputpath = !pwd

In [8]:
# save reconstructed image
cv2.imwrite(os.path.join(outputpath[0], 'reconstructed.png'), reconstruct_I)

True